# Исследовательский анализ данных (EDA) датасета Titanic

Цель анализа: оценить структуру, полноту и качество данных, выявить возможные выбросы и аномалии для последующей подготовки данных к моделированию.


In [ ]:
import pandas as pd
import numpy as np

FILE_ID = "1Tj8RQHUh7MTmE88paqDBqDJZ5Ru6mOtL"
file_url = f"https://drive.google.com/uc?export=download&id={FILE_ID}"

# Загрузка датасета с учетом кавычек и пробелов
df_raw = pd.read_csv(file_url, sep=",", quotechar='"', skipinitialspace=True, encoding="utf-8")

print(f"Размер датасета: {df_raw.shape[0]} строк, {df_raw.shape[1]} столбцов")
print("\nПервые 10 строк:")
display(df_raw.head(10))

In [ ]:
df = df_raw.copy()

# Приведение к числовым типам (Int32 nullable и Float32)
int_cols = ["PassengerId", "Survived", "Pclass", "SibSp", "Parch"]
for col in int_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int32")

float_cols = ["Age", "Fare"]
for col in float_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce").astype("Float32")

# Текстовые и категориальные признаки
df["Name"] = df["Name"].astype("string")
cat_cols = ["Sex", "Ticket", "Cabin", "Embarked"]
for col in cat_cols:
    df[col] = df[col].astype("string").astype("category")

print("\nТипы данных после приведения:")
print(df.dtypes)

In [ ]:
print("\nОбщая информация по датасету:")
df.info()

print("\nКоличество уникальных значений в категориальных столбцах:")
for col in cat_cols:
    unique_vals = df[col].nunique()
    print(f" - {col}: {unique_vals}")

print("\nОсновные статистики числовых характеристик:")
display(df.describe())

In [ ]:
print("\nПропущенные значения по столбцам:")
missing_counts = df.isna().sum()
missing_percent = (missing_counts / len(df)) * 100
missing_df = pd.DataFrame({"count_missing": missing_counts, "percent_missing": missing_percent})
display(missing_df)

print("Вывод: Колонки с наибольшим процентом пропусков требуют внимания при обработке.")

In [ ]:
def find_outliers_iqr(data, col):
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    return outliers

num_cols = int_cols + float_cols
print("\nВыбросы в числовых признаках:")
for col in num_cols:
    outliers = find_outliers_iqr(df, col)
    print(f" - {col}: {len(outliers)} выбросов ({len(outliers)/len(df)*100:.2f} % от данных)")

Выводы:
- Датасет содержит 891 строку и 12 столбцов.
- Есть пропуски в столбцах Age, Cabin и Embarked, наибольшие пропуски в Cabin (~77%).
- Выбросы выявлены в числовых столбцах Fare, Age и других; требует дальнейшего анализа и решения.
- Типы данных приведены к оптимальным для дальнейшей работы.

Рекомендуется:
- Обработать пропуски в Age и Embarked, возможно, заменить или удалить.
- Рассмотреть удаление или кодирование Cabin из-за большого количества пропусков.
- Учесть выбросы при построении модели.

In [ ]:
# Пример - Создание бинарного признака "IsAlone" по SibSp и Parch
df['IsAlone'] = ((df['SibSp'] + df['Parch']) == 0).astype('Int32')
print("\nДобавлен бинарный признак IsAlone.")
display(df[['SibSp', 'Parch', 'IsAlone']].head())

# Корреляционная матрица числовых признаков (без сложной визуализации)
print("\nКорреляционная матрица:")
corr_matrix = df[num_cols + ['IsAlone']].corr()
display(corr_matrix)